In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
num_epochs = 5
batch_size = 40
learning_rate = 0.001
classes = ('plane', 'car' , 'bird',
    'cat', 'deer', 'dog',
    'frog', 'horse', 'ship', 'truck')

cuda


In [3]:
transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize( 
       (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010) 
    )
])
train_dataset = torchvision.datasets.CIFAR10(
    root= './data', train = True,
    download =True, transform = transform)
test_dataset = torchvision.datasets.CIFAR10(
    root= './data', train = False,
    download =True, transform = transform)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
train_loader = torch.utils.data.DataLoader(train_dataset
    , batch_size = batch_size
    , shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset
    , batch_size = batch_size
    , shuffle = True)
n_total_step = len(train_loader)
print(n_total_step)

1250


In [5]:
model = models.vgg16(pretrained = True)
input_lastLayer = model.classifier[6].in_features
model.classifier[6] = nn.Linear(input_lastLayer,10)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum=0.9,weight_decay=5e-4)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


In [9]:
for epoch in range(num_epochs):
    for i, (imgs , labels) in enumerate(train_loader):
        imgs = imgs.to(device)
        labels = labels.to(device)

        labels_hat = model(imgs)
        n_corrects = (labels_hat.argmax(axis=1)==labels).sum().item()
        loss_value = criterion(labels_hat, labels)
        loss_value.backward()
        optimizer.step()
        optimizer.zero_grad()
        if (i+1) % 250 == 0:
           print(f'epoch {epoch+1}/{num_epochs}, step: {i+1}/{n_total_step}: loss = {loss_value:.5f}, acc = {100*(n_corrects/labels.size(0)):.2f}%')
    print()

epoch 1/5, step: 250/1250: loss = 0.44237, acc = 90.00%
epoch 1/5, step: 500/1250: loss = 0.36418, acc = 90.00%
epoch 1/5, step: 750/1250: loss = 0.18683, acc = 90.00%
epoch 1/5, step: 1000/1250: loss = 0.23285, acc = 87.50%
epoch 1/5, step: 1250/1250: loss = 0.25706, acc = 90.00%

epoch 2/5, step: 250/1250: loss = 0.47374, acc = 87.50%
epoch 2/5, step: 500/1250: loss = 0.33710, acc = 87.50%
epoch 2/5, step: 750/1250: loss = 0.18075, acc = 92.50%
epoch 2/5, step: 1000/1250: loss = 0.11581, acc = 97.50%
epoch 2/5, step: 1250/1250: loss = 0.45148, acc = 82.50%

epoch 3/5, step: 250/1250: loss = 0.05908, acc = 97.50%
epoch 3/5, step: 500/1250: loss = 0.08551, acc = 97.50%
epoch 3/5, step: 750/1250: loss = 0.06765, acc = 97.50%
epoch 3/5, step: 1000/1250: loss = 0.05619, acc = 97.50%
epoch 3/5, step: 1250/1250: loss = 0.04385, acc = 100.00%

epoch 4/5, step: 250/1250: loss = 0.09301, acc = 95.00%
epoch 4/5, step: 500/1250: loss = 0.11216, acc = 95.00%
epoch 4/5, step: 750/1250: loss = 0.17

In [11]:
with torch.no_grad():
    number_corrects = 0
    number_samples = 0
    for i, (test_images_set , test_labels_set) in enumerate(test_loader):
        test_images_set = test_images_set.to(device)
        test_labels_set = test_labels_set.to(device)
    
        y_predicted = model(test_images_set)
        labels_predicted = y_predicted.argmax(axis = 1)
        number_corrects += (labels_predicted==test_labels_set).sum().item()
        number_samples += test_labels_set.size(0)
    print(f'Overall accuracy {(number_corrects / number_samples)*100}%')

Overall accuracy 92.52%
